# Chatbot using char embedding and an LSTM layer
- Doesn't require exact words defined in intents.json (char tokenizing vs word tokenizing)
- Tolerant to typos and words not found in intents.json
- Slower training and predicting times compared to using dense layers or word tokenization
- Better recognition of unknown phrases

# Setup

In [1]:
import numpy as np
import random
import json
import nltk
import tensorflow as tf
from tensorflow import keras
import pickle

# Read and parse intents.json

In [21]:
with open('intents.json') as file:
    data = json.load(file)

words = []
labels = []
docs_x = []
docs_y = []

tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)

for intent in data['intents']:
    for pattern in intent['patterns']:
        wrds = nltk.word_tokenize(pattern)
        wrds = [w.lower() for w in wrds if w != '?']
        words.extend(wrds)
        docs_x.append(' '.join(wrds))
        docs_y.append(intent['tag'])
        
    if intent['tag'] not in labels:
        labels.append(intent['tag'])


words = sorted(list(set(words)))
tokenizer.fit_on_texts(' '.join(words))

labels = sorted(labels)

training = []
output = []

out_empty = [0 for _ in range(len(labels))]
training_empty = np.array([0. for _ in range(len(tokenizer.word_index)+1)])

for x, doc in enumerate(docs_x):
    
#     doc_onehot_sequence = tokenizer.texts_to_matrix(doc)
    doc_sequence = np.array(tokenizer.texts_to_sequences(doc))
    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])] = 1
    
#     training.append(doc_onehot_sequence)
    training.append(doc_sequence)
    output.append(output_row)
    
training_padded = keras.preprocessing.sequence.pad_sequences(training)
output = np.array(output)

# Define model and train (this time with embedding matrix and an LSTM layer)

In [3]:
model = keras.models.Sequential([
    
    keras.layers.Embedding(len(tokenizer.word_index)+1,8, mask_zero=True),
    keras.layers.LSTM(16),
    keras.layers.Dense(len(labels), activation='softmax')
    
])

In [ ]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics='accuracy')
history = model.fit(x=training_padded,
                    y=output,
                    batch_size=8,
                    epochs=1000,
                    callbacks=[keras.callbacks.ReduceLROnPlateau(monitor='loss',factor=0.7,patience=30),
                               keras.callbacks.EarlyStopping(monitor='loss', patience=150, restore_best_weights=True)])

# Setup predictions

In [6]:
def str_to_sequence(inp):
    wrds = nltk.word_tokenize(inp)
    wrds = [w.lower() for w in wrds if w != '?']
    inp_text = ' '.join(wrds)
    inp_seq = tokenizer.texts_to_sequences(inp_text)
    return np.array(inp_seq).reshape(1,-1,1)

def model_response(pred, labels, data):
    if max(pred)<0.9:
        uncertain_responses = ['I dont understand', 'Sorry I dont get what you\'re saying', 'Could you word that differently?']
        print(f"Chatbot: {random.choice(uncertain_responses)}")
    else:
        pred_idx = np.argmax(pred)
        label = labels[pred_idx]

        for intent in data['intents']:
            if label == intent['tag']:
                print(f"Chatbot: {random.choice(intent['responses'])}")
                break

# Define chat loop and chat!

In [12]:
def chat():
    print('Start talking with the bot! (type quit to stop)\n')
    while True:
        inp = input("You: ")
        if inp.lower()=='quit':
            break
        inp = ''.join([c for c in inp if c in tokenizer.word_index.keys()])
        inp_seq = str_to_sequence(inp)
        pred = model.predict(inp_seq).reshape(-1)
        model_response(pred,labels,data)

In [23]:
chat()

Start talking with the bot! (type quit to stop)

You: hello!
Chatbot: Good to see you again!
You: how are you?
Chatbot: Good to see you again!
You: what do you sell?
Chatbot: We sell chocolate chip cookies for $2!
You: do you have a name?
Chatbot: I'm Chatbot!
You: how old are you?
Chatbot: I am 18 years old!
You: do you deliver?
Chatbot: Sure, call this number and make your delivery!
You: hellwo!
Chatbot: Hi there, how can I help?
You: do u make reservation
Chatbot: I dont understand
You: do u accept reservation
Chatbot: We'd gladly take your reservation, call this number
You: wat is ur name
Chatbot: I'm Chatbot!
You: goodby chatbot!
Chatbot: Goodbye!
You: quit
